In [1]:
# coding=utf-df
import pandas as pd
import numpy as np

import Input
from genCorpus import CutContent, TrainPosting, TrainResume, MyCorpus
import featsettings
import featutils
import ngram

from featutils import try_divide, dump_feat_name

train_post = Input.Input.from_name('TrainPosting')  # 17483
train_resume = Input.Input.from_name('TrainResume')  # 86312
postCutContent = CutContent(train_post)
postCutTitle = CutContent(train_post, 2)
resumeCutContent = CutContent(train_resume)

D:\ProgramFiles\Anaconda\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Loading model from cache c:\users\mac\appdata\local\temp\jieba.cache
DEBUG:jieba:Loading model from cache c:\users\mac\appdata\local\temp\jieba.cache
Loading model cost 0.369 seconds.
DEBUG:jieba:Loading model cost 0.369 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


[<class 'genCorpus.TrainPosting'>, <class 'genCorpus.TrainResume'>]
[<class 'genCorpus.TrainPosting'>, <class 'genCorpus.TrainResume'>]


In [2]:
qc_apply_info = pd.read_csv('qc_data', names=['f_post_id', 'f_resume_id', 'date', 'state', 'seq_type', 'rank_score'])
map_dic = {3:1,1:0}
qc_apply_info['rank_score'] = map(lambda x: map_dic[x], qc_apply_info.rank_score)

In [3]:
qc_apply_info.f_post_id = qc_apply_info.f_post_id.astype(np.string_)
qc_apply_info.f_resume_id = qc_apply_info.f_resume_id.astype(np.string_)

In [4]:
qc_apply_info['posts'] = list(qc_apply_info[['f_post_id']].apply(lambda x:  postCutContent[x['f_post_id']], axis=1))

In [5]:
qc_apply_info.head()

,f_post_id,f_resume_id,date,state,seq_type,rank_score,posts
0,950725096,13293,2015-09-11,终面通过拟录用审核,其他正式,1,"[本科, 以上学历, 以上, 用户, 运营, 经验, 优先, 了解, 用户, 思维, 方式,..."
1,220920696,31091,2014-07-03,已离职,P正式,1,"[两年, 以上, 工作, 经验, 熟练, 使用, c++, 熟悉, x86, 汇编语言, 熟..."
2,159463555,33286,2013-09-22,已离职,T正式,1,"[计算机科学, 相关, 毕业, 以上, 工作, 经验, 精通, javascript, aj..."
3,255695530,87956,2014-09-01,已离职,其他正式,1,"[从事, 无线, bd, 工作, 两年, 以上, 海外, 无线, 推广, 深刻, 认识, 广..."
4,4104,99350,2013-02-20,待入职,T正式,1,"[本科, 以上学历, 计算机软件, 通讯, 相关, 专业, 优先, 至少, 相关, 工作, ..."


In [6]:
qc_apply_info['resumes'] = list(qc_apply_info[['f_resume_id']].apply(lambda x:  resumeCutContent[x['f_resume_id']], axis=1))

In [7]:
qc_apply_info.head()

,f_post_id,f_resume_id,date,state,seq_type,rank_score,posts,resumes
0,950725096,13293,2015-09-11,终面通过拟录用审核,其他正式,1,"[本科, 以上学历, 以上, 用户, 运营, 经验, 优先, 了解, 用户, 思维, 方式,...","[工作, 描述, 公司, 增值, 产品, 收入, 负责, 负责, 增值, 产品, 项目, 策..."
1,220920696,31091,2014-07-03,已离职,P正式,1,"[两年, 以上, 工作, 经验, 熟练, 使用, c++, 熟悉, x86, 汇编语言, 熟...","[修复, 虚拟, 桌面, soda, sym, ant, ec, demand, agent..."
2,159463555,33286,2013-09-22,已离职,T正式,1,"[计算机科学, 相关, 毕业, 以上, 工作, 经验, 精通, javascript, aj...","[尚品, 前端, 研发, 经理, 汇报, 对象, 研发, 中心, 高级, 总监, 负责, 网..."
3,255695530,87956,2014-09-01,已离职,其他正式,1,"[从事, 无线, bd, 工作, 两年, 以上, 海外, 无线, 推广, 深刻, 认识, 广...","[2001, 4000, 公司, 产品, 网络, 推广, 以及, 电子商务, 平台, 维护,..."
4,4104,99350,2013-02-20,待入职,T正式,1,"[本科, 以上学历, 计算机软件, 通讯, 相关, 专业, 优先, 至少, 相关, 工作, ...","[win32, 操作系统, windows, nt, windows, mobile, 创建..."


In [9]:
import gensim
from gensim.corpora.dictionary import Dictionary

# postDictionary = Dictionary(dict(zip(qc_apply_info['f_post_id'], qc_apply_info['posts'])).values())
# resumeDictionary = Dictionary(dict(zip(qc_apply_info['f_resume_id'], qc_apply_info['resumes'])).values())
# overallDictionary = Dictionary(dict(zip(qc_apply_info['f_post_id'], qc_apply_info['posts'])).values()+
#                               dict(zip(qc_apply_info['f_resume_id'], qc_apply_info['resumes'])).values())

# all_texts = reduce(lambda x, y: x+y, dict(zip(qc_apply_info['f_post_id'], qc_apply_info['posts'])).values()+
#                               dict(zip(qc_apply_info['f_resume_id'], qc_apply_info['resumes'])).values())

# all_texts = [word for word in line for line in list(qc_apply_info.posts)+list(qc_apply_info.resumes)]


all_texts =  list(qc_apply_info.posts)+list(qc_apply_info.resumes)
texts_words = [word for line in all_texts for word in line ]

In [11]:
texts_words, all_texts

([u'\u672c\u79d1',
  u'\u4ee5\u4e0a\u5b66\u5386',
  u'\u4ee5\u4e0a',
  u'\u7528\u6237',
  u'\u8fd0\u8425',
  u'\u7ecf\u9a8c',
  u'\u4f18\u5148',
  u'\u4e86\u89e3',
  u'\u7528\u6237',
  u'\u601d\u7ef4',
  u'\u65b9\u5f0f',
  u'\u7528\u6237',
  u'\u89d2\u5ea6',
  u'\u601d\u8003',
  u'\u89e3\u51b3\u95ee\u9898',
  u'\u5584\u4e8e',
  u'\u6c9f\u901a',
  u'\u7ec4\u7ec7',
  u'\u534f\u8c03',
  u'\u826f\u597d',
  u'\u6267\u884c\u529b',
  u'\u826f\u597d',
  u'\u5b66\u4e60',
  u'\u80fd\u529b',
  u'\u56e2\u961f',
  u'\u5408\u4f5c',
  u'\u7cbe\u795e',
  u'\u4e24\u5e74',
  u'\u4ee5\u4e0a',
  u'\u5de5\u4f5c',
  u'\u7ecf\u9a8c',
  u'\u719f\u7ec3',
  u'\u4f7f\u7528',
  u'c++',
  u'\u719f\u6089',
  u'x86',
  u'\u6c47\u7f16\u8bed\u8a00',
  u'\u719f\u7ec3\u638c\u63e1',
  u'\u5e38\u89c1',
  u'\u6570\u636e\u7ed3\u6784',
  u'\u7b97\u6cd5',
  u'\u53ef\u4ee5',
  u'\u8fd0\u7528',
  u'\u5e38\u89c1',
  u'\u6570\u636e\u7ed3\u6784',
  u'\u7b97\u6cd5',
  u'\u89e3\u51b3',
  u'\u9879\u76ee',
  u'\u5b9e\u9645',
  u'\u95e

In [12]:
dicwords = set(pd.Series(texts_words).value_counts(ascending=False)[:1000].index.tolist())
new_corpus = [ ]

for line in all_texts:
    temp_line = []
    for word in line:
        if word in dicwords:
            temp_line.append(word)
    new_corpus.append(temp_line)

# dict1 = copy.deepcopy(postDictionary)
# dict2 = copy.deepcopy(resumeDictionary)
# # 7862
# dict1.filter_extremes(no_below=5, no_above=0.5, keep_n=100000, keep_tokens=None)
# # 5 63389 | 10 41352  | 50 15450
# dict2.filter_extremes(no_below=50, no_above=0.4, keep_n=100000, keep_tokens=None)
# dict2_to_dict1 = dict1.merge_with(dict2)
overall_dict = Dictionary(new_corpus)

In [13]:
# gensim.matutils.corpus2dense
qc_apply_info['posts_bow'] = map(lambda x: overall_dict.doc2bow(x),qc_apply_info['posts'] )
qc_apply_info['resumes_bow'] = map(lambda x: overall_dict.doc2bow(x),qc_apply_info['resumes'] )

In [82]:

import gensim
dPosts = gensim.matutils.corpus2dense(qc_apply_info['posts_bow'].tolist(), 1000).T
dResumes = gensim.matutils.corpus2dense(qc_apply_info['posts_bow'].tolist(), 1000).T
Y_train= np.array(qc_apply_info.rank_score.tolist())

# X_train = np.hstack[[dPosts, dResumes]]

# X_test = X_train[-X_train.shape[0]/10:]
# Y_test = Y_train[-X_train.shape[0]/10:]
# X_train = X_train[:-X_train.shape[0]/10]
# Y_train = Y_train[:-X_train.shape[0]/10]

In [83]:
import numpy as np
X_train = np.hstack([dPosts, dResumes])

In [84]:
X_train, Y_train, X_train.shape, np.array(Y_train).shape

(array([[ 1.,  1.,  1., ...,  0.,  0.,  0.],
        [ 0.,  1.,  1., ...,  0.,  0.,  0.],
        [ 0.,  1.,  1., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  1., ...,  0.,  0.,  0.],
        [ 0.,  1.,  3., ...,  0.,  0.,  0.],
        [ 1.,  2.,  1., ...,  0.,  0.,  0.]], dtype=float32),
 array([1, 1, 1, ..., 0, 0, 0], dtype=int64),
 (16749L, 2000L),
 (16749L,))

In [85]:
split_point = X_train.shape[0]/10
X_test = X_train[-split_point:]
Y_test = Y_train[-split_point:]
X_train = X_train[:-split_point]
Y_train = Y_train[:-split_point]

In [86]:
X_test, Y_test, X_train,Y_train,X_test.shape, Y_test.shape, X_train.shape,Y_train.shape
from copy import deepcopy
X_total = deepcopy(X_train)
Y_total = deepcopy(Y_train)

In [93]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=3)
for train_index, test_index in skf.split(X_total, Y_total):
    print train_index, test_index

[ 2760  2761  2762 ..., 15072 15073 15074] [    0     1     2 ..., 10542 10543 10544]
[    0     1     2 ..., 15072 15073 15074] [ 2760  2761  2762 ..., 12807 12808 12809]
[    0     1     2 ..., 12807 12808 12809] [ 5520  5521  5522 ..., 15072 15073 15074]


In [94]:
X_total.shape, Y_total.shape

((15075L, 2000L), (15075L,))

In [98]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=3)
from sklearn.ensemble import RandomForestClassifier

Y_total = np.array(Y_total)
for train_index, test_index in skf.split(X_total, Y_total):
    X_train, X_valid = X_total[train_index], X_total[test_index]
    Y_train, Y_valid = Y_total[train_index], Y_total[test_index]
    print Y_total[train_index].shape
    for n_estimators in range(50, 500, 50):
        clf = RandomForestClassifier(n_estimators=n_estimators)
        clf.fit(np.asarray(X_train, dtype="float32"), Y_train)
        # set_trace()
#         predict_validation = clf.predict_proba(np.asarray(X_valid, dtype="float32"))
        print 'n_estimators  ',   n_estimators
        print X_test.shape, Y_test.shape, X_train.shape,Y_train.shape, X_valid.shape, y_valid.shape
        print clf.score(X_train, Y_train), clf.score(X_valid, y_valid), clf.score(X_test, Y_test)

(10050L,)
n_estimators   50
(1674L, 2000L) (1674L,) (10050L, 2000L) (10050L,) (5025L, 2000L) (5025L,)
0.64447761194 0.344875621891 0.106929510155
n_estimators   100
(1674L, 2000L) (1674L,) (10050L, 2000L) (10050L,) (5025L, 2000L) (5025L,)
0.644676616915 0.345074626866 0.10394265233
n_estimators   150
(1674L, 2000L) (1674L,) (10050L, 2000L) (10050L,) (5025L, 2000L) (5025L,)
0.644776119403 0.344278606965 0.104540023895
n_estimators   200
(1674L, 2000L) (1674L,) (10050L, 2000L) (10050L,) (5025L, 2000L) (5025L,)
0.644776119403 0.346666666667 0.0991636798088
n_estimators   250
(1674L, 2000L) (1674L,) (10050L, 2000L) (10050L,) (5025L, 2000L) (5025L,)
0.644776119403 0.343880597015 0.108124253286
n_estimators   300
(1674L, 2000L) (1674L,) (10050L, 2000L) (10050L,) (5025L, 2000L) (5025L,)
0.644776119403 0.348855721393 0.0919952210275
n_estimators   350
(1674L, 2000L) (1674L,) (10050L, 2000L) (10050L,) (5025L, 2000L) (5025L,)
0.644776119403 0.345074626866 0.103345280765
n_estimators   400
(1674L

KeyboardInterrupt: 